# Supervised machine learning: Introduction and regularization 
## Binary classification with text data

# Imports

In [22]:
## load packages 
import pandas as pd
import re
import numpy as np
import plotnine
from plotnine import *
import pickle

## nltk imports
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

## print mult things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## random
import random

pd.set_option('display.max_colwidth', None)

In [23]:
## function to process text
def processtext(one_str, stop_list):
    
    ## remove stopwords
    no_stop = [tok for tok in wordpunct_tokenize(one_str)
              if tok not in stop_list]
    
    
    processed_string = " ".join([porter.stem(i.lower()) 
                        for i in no_stop if 
                        i.lower().isalpha() and len(i) >=3])
    return(processed_string)

## function to create dtm
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

# Load data

Load labeled yelp data in `public_data` and run below code

**Note**: make sure to change your path if you need to; if you're having trouble loading the `pkl`, try running on jupyter hub since it may be a python versioning issue

In [24]:
# If you have trouble loading these data (kernel dies due to memory issues), try sampling down to 5000 or 1000 rows
yelp = pd.read_pickle("../public_data/yelp_forML.pkl") #.sample(n=1000)

In [25]:
## preprocess data to create dtm
porter = PorterStemmer()
list_stopwords = stopwords.words("english")

yelp['process_text'] = [processtext(one_review, stop_list = list_stopwords) 
                        for one_review in yelp['raw_text']]

yelp_dtm = create_dtm(yelp['process_text'], yelp[['metadata_label', 'metadata_rowid',
                                                 'process_text', 'raw_text']])

# 1. Split into features, labels, and split into training/hold out

## 1.1 Split into X (features or id metadata) and y (labels)

In [26]:
X = yelp_dtm[[col for col in yelp_dtm.columns if col not in ['metadata_label',
                                                            'index']]].copy()
y = yelp_dtm[['metadata_label']]

In [27]:
## checking dimensionality
X.shape
y.shape

assert X.shape[0] == y.shape[0]
assert y.shape[1] == 1


(15000, 23439)

(15000, 1)

## 1.2 using automatic function to create train-test split

In [28]:
### using built-in function
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.2,
                                                   random_state = 221)

In [30]:

y_test.values.ravel()

array([0, 1, 0, ..., 1, 1, 0])

## 1.3 using more manual approach to create train-test split

In [31]:
### more manually: useful when we want more control
### over the ids (eg clustering or time ordering)
### or if we want to go back to matrix before preprocessing
nrows_train = round(X.shape[0]*0.8)
nrows_test = X.shape[0] - nrows_train
random.seed(221)
train_ids = random.sample(list(set(X['metadata_rowid'])), nrows_train)

def my_split(X, y, 
             train_ids, 
             id_col):
    
    ## get test ids
    test_ids = set(X[id_col]).difference(train_ids)
    
    ## split
    X_train_man = X[X[id_col].isin(train_ids)].copy()
    X_test_man = X[X[id_col].isin(test_ids)].copy()
    y_train_man = y[y.index.isin(train_ids)].iloc[:, 0].to_numpy()
    y_test_man = y[y.index.isin(test_ids)].iloc[:, 0].to_numpy()
    
    ## return
    return(X_train_man, X_test_man, y_train_man, y_test_man)

In [32]:
X_train_man, X_test_man, y_train_man, y_test_man = my_split(X, y,
                                                            train_ids, 
                                                            id_col = 'metadata_rowid')

In [33]:
y_test_man

array([0, 0, 1, ..., 0, 0, 0])

# 2. Estimate models with hardcoded parameters: logistic regression with L1 regularization (Lasso)

## 2.1 Estimate model using training data

In [34]:

non_feat = ['metadata_rowid', 'raw_text', 'process_text']
logit_lasso = LogisticRegression(penalty = "l1",max_iter=100, 
             C = 0.01, solver='liblinear')
logit_lasso.fit(X_train_man[[col for col in X_train.columns if col not in 
                   non_feat]], y_train_man)

LogisticRegression(C=0.01, penalty='l1', solver='liblinear')

## 2.2 Generate predictions in test data

In [35]:
y_pred = logit_lasso.predict(X_test_man[[col for col 
                in X_test_man.columns if col not in non_feat]])
y_predprob = logit_lasso.predict_proba(X_test_man[[col for col 
                in X_test_man.columns if col not in non_feat]])

In [12]:
## print the results 
y_pred[0:5]
y_predprob[0:5]


array([0, 0, 1, 0, 0])

array([[0.64279266, 0.35720734],
       [0.69705267, 0.30294733],
       [0.06367281, 0.93632719],
       [0.72128942, 0.27871058],
       [0.50402252, 0.49597748]])

## 2.3 Clean up predictions and calculate error metrics

In [36]:
## make into a dataframe
y_pred_df = pd.DataFrame({'y_pred_binary': y_pred,
                         'y_pred_continuous': [one_prob[1] 
                                            for one_prob in y_predprob],
                         'y_true': y_test_man})
y_pred_df.sample(n = 10, random_state = 4484)
y_pred_df.head()


,y_pred_binary,y_pred_continuous,y_true
656,0,0.240226,0
825,1,0.698157,1
1641,1,0.790618,1
2249,1,0.535206,1
1080,0,0.448078,0
2490,0,0.432704,1
2857,1,0.679533,1
1160,1,0.681760,1
1061,1,0.519448,0
2255,0,0.193009,0


,y_pred_binary,y_pred_continuous,y_true
0,0,0.357153,0
1,0,0.302924,0
2,1,0.936390,1
3,0,0.278692,0
4,0,0.495900,0


In [37]:
## precision as tp / tp+fp 
error_cond = [(y_pred_df['y_true'] == 1) & (y_pred_df['y_pred_binary'] == 1),
             (y_pred_df['y_true'] == 1) & (y_pred_df['y_pred_binary'] == 0),
              (y_pred_df['y_true'] == 0) & (y_pred_df['y_pred_binary'] == 0)]

error_codeto = ["TP", "FN", "TN"]

y_pred_df['error_cat'] = np.select(error_cond, error_codeto, default = "FP")
y_error = y_pred_df.error_cat.value_counts().reset_index().copy()
y_error.columns = ['cat', 'n']
y_error

### precision
print("Precision is:-----------")
y_error.loc[y_error.cat == "TP", 'n'].iloc[0]/(y_error.loc[y_error.cat == "TP", 'n'].iloc[0] +
                    y_error.loc[y_error.cat == "FP", 'n'].iloc[0])

### recall
print("Recall is:---------------")
y_error.loc[y_error.cat == "TP", 'n'].iloc[0]/(y_error.loc[y_error.cat == "TP", 'n'].iloc[0] +
                    y_error.loc[y_error.cat == "FN", 'n'].iloc[0])

,cat,n
0,TN,1320
1,TP,1103
2,FN,319
3,FP,258


Precision is:-----------


0.8104335047759

Recall is:---------------


0.7756680731364276

## 2.4 Interpret the model

In [38]:
## get top features
las_coef = pd.DataFrame({'coef': logit_lasso.coef_[0],
                         'feature_name': 
                        [col for col in X_train.columns if col not in non_feat]})

#las_coef.sort_values(by = 'coef', ascending = False)


top_feat = las_coef.sort_values(by = 'coef', ascending = False)[0:10]
top_feat_list = top_feat.feature_name.to_list()

all_agg = [yelp_dtm.groupby(['metadata_label']).agg({one_feat: np.mean})
for one_feat in top_feat_list]

all_agg_df = pd.concat(all_agg, axis = 1)
all_agg_df

/var/folders/0f/r6sb0yjx70l_25cgtvyjjqgr0000gn/T/ipykernel_59921/1319144360.py:12: FutureWarning: The provided callable <function mean at 0x10a5a6700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.


,delici,great,love,amaz,excel,best,favorit,friendli,definit,alway
metadata_label,,,,,,,,,,
0,0.026424,0.171178,0.085976,0.022944,0.018819,0.103506,0.027713,0.067672,0.059036,0.113302
1,0.162248,0.557305,0.284452,0.104529,0.093621,0.219691,0.116819,0.169014,0.142640,0.242613


# 3. Compare performance across hyperparameters for logistic regression

Would our logit model make more accurate predictions if we fed it different hyperpameters? Which hyperparameters would be the best? Let's find out.

1. Define a function that:
- takes in a cost parameter (*C*, the inverse of regularization strength)
- trains a logistic regression model with L1 regularization (Lasso) and otherwise has the same parameters as above
- fits the model on the training data
- makes predictions and returns them as a DataFrame

2. Use the function to get predictions for the list of *C* parameters below, then bind them into one DataFrame.

3. Finally, score the precision for each model (each iteration of *C*) and show which model scores the best.

**Hint**: To compute precision score, you can use:
```python
precision_score(
    one_df['y_true'], one_df['y_pred'],
    zero_division = 0) # silences warning
```

In [ ]:
# Provided set of hyperparameters on which to train and then compare performance
c_list = np.linspace(4, 0.0001, 5)

In [ ]:
## Solution:
## define function that takes in one cost parameter
## and estimates model, returning pred
def one_las(C):
    one_lasso = LogisticRegression(penalty = "l1", max_iter=100, 
             C = C, solver='liblinear')
    
    one_lasso.fit(X_train_man[[col for col in X_train.columns if 
                              col not in non_feat]], y_train_man)
    
    y_pred = one_lasso.predict(X_test_man[[col for col in X_test_man.columns 
                if col not in non_feat]])
    
    y_pred_df = pd.DataFrame({'y_pred': y_pred, 
                             'y_true': y_test_man,
                             'cost': C})
    return(y_pred_df)

In [ ]:
for c in c_list: one_las(c)

In [ ]:
## make into a dataframe
y_pred_df_1 = pd.DataFrame({'y_pred_binary': y_pred,
                         'y_pred_continuous': [one_prob[1] 
                                            for one_prob in y_predprob],
                         'y_true': y_test_man})
y_pred_df_1.sample(n = 10, random_state = 4484)
y_pred_df_1.head()

In [ ]:
## precision as tp / tp+fp 
error_cond = [(y_pred_df_1['y_true'] == 1) & (y_pred_df_1['y_pred_binary'] == 1),
             (y_pred_df_1['y_true'] == 1) & (y_pred_df_1['y_pred_binary'] == 0),
              (y_pred_df_1['y_true'] == 0) & (y_pred_df_1['y_pred_binary'] == 0)]

error_codeto = ["TP", "FN", "TN"]

y_pred_df_1['error_cat'] = np.select(error_cond, error_codeto, default = "FP")
y_error = y_pred_df_1.error_cat.value_counts().reset_index().copy()
y_error.columns = ['cat', 'n']
y_error

### precision
print("Precision is:-----------")
y_error.loc[y_error.cat == "TP", 'n'].iloc[0]/(y_error.loc[y_error.cat == "TP", 'n'].iloc[0] +
                    y_error.loc[y_error.cat == "FP", 'n'].iloc[0])

### recall
print("Recall is:---------------")
y_error.loc[y_error.cat == "TP", 'n'].iloc[0]/(y_error.loc[y_error.cat == "TP", 'n'].iloc[0] +
                    y_error.loc[y_error.cat == "FN", 'n'].iloc[0])

# 4. Activity 

- Read the documentation here to initialize a ridge regression (l2 penalty)- you can use the same cost parameter (C) and number of iterations as in the lasso example above: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- Fit the model on X_train_man, y_train_main 
- Generate binary and continuous predictions
- Create a function that takes in a dataframe of binary predictions and true labels and manually calculates the $F_{1}$ score:

$$F_{1} = 2 * \dfrac{precision * recall}{precision + recall} = \dfrac{TP}{TP + 0.5(FP + FN)}$$

- Apply that function to calculate the F1 score for the decision tree and lasso (from above), and ridge regression (from the activity)
- *Challenge exercise*: parametrize the model fitting with a function that takes in a classifier as an argument and returns coefficients or feature importances and certain eval metrics (eg precision, recall, and F1)

In [39]:
# your code here 
logit_ridge = LogisticRegression(penalty = "l2",max_iter=100, 
             C = 1, solver='liblinear')

X_feat_tr = X_train_man[[col for col in X_train_man.columns 
                if col not in non_feat]]
X_feat_te = X_test_man[[col for col in X_test_man.columns 
                if col not in non_feat]]
logit_ridge.fit(X_feat_tr, y_train_man)
## predict
y_hat = logit_ridge.predict(X_feat_te)
    


LogisticRegression(C=1, solver='liblinear')

In [40]:
def calc_myF1(fit_classifier, X_test, y_test):
    
    ## predict
    y_hat = fit_classifier.predict(X_test)
    
    ## get the relevant counts
    df_pred = pd.DataFrame({'y_true': y_test,
                           'y_pred_binary': y_hat})
    
    ## counts of diff metrics
    tp = df_pred[(df_pred.y_true == 1) &
            (df_pred.y_pred_binary == 1)].shape[0]
    fp = df_pred[(df_pred.y_true == 0) &
            (df_pred.y_pred_binary == 1)].shape[0]
    fn = df_pred[(df_pred.y_true == 1) &
            (df_pred.y_pred_binary == 0)].shape[0]
    
    ## combine
    f1 = (tp)/(tp + 0.5*(fp + fn))
    
    ## return
    return(f1)


In [42]:
calc_myF1(logit_ridge, X_test = X_feat_te, y_test = y_test_man)
calc_myF1(logit_lasso, X_test = X_feat_te, y_test = y_test_man)


0.8745220716023636

0.7926697808120733

In [43]:
ridge_c = pd.DataFrame({'coef': logit_ridge.coef_[0],
                        'feat': X_feat_tr.columns})

ridge_c[ridge_c.coef != 0].head()
ridge_c.sort_values(by = 'coef', ascending = False)

,coef,feat
0,0.005291,aaa
1,-0.001103,aaaaaa
2,0.000004,aaaaaaaaaaaaaaand
3,0.000129,aaaaaaaaaaaaahhhhhhhhhhh
4,-0.076687,aaaaand


,coef,feat
5009,1.862923,delici
526,1.815087,amaz
6744,1.800407,fantast
19359,1.766226,solid
6510,1.654128,excel
...,...,...
1880,-2.064684,bland
11696,-2.122132,mediocr
20768,-2.160225,terribl
23108,-2.437169,worst


In [44]:
# y_test_man